In [15]:
#Get the speech to text command. 
import sounddevice as sd
import numpy as np
import whisper
from scipy.io.wavfile import write

# Step 1: Record audio from microphone
fs = 44100  # Sample rate
seconds = 30  # Duration of recording
print("Recording... Speak now!")
recording = sd.rec(int(seconds * fs), samplerate=fs, channels=1, dtype=np.int16)
sd.wait()
wav_filename = "recorded_audio.wav"
write(wav_filename, fs, recording)
print(f"Recording saved as '{wav_filename}'.")

# Step 2: Transcribe the audio using Whisper
model = whisper.load_model("base")  # Use "tiny", "small", "medium", or "large" as needed
result = model.transcribe(wav_filename)

# Step 3: Print the transcription
print("Transcription:")
print(result["text"])
message=result["text"]

Recording... Speak now!
Recording saved as 'recorded_audio.wav'.


/Users/hjiang/miniforge3/envs/ai-agent/lib/python3.11/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcription:
 Please create a new card under to do titled AI Productivity and in the description say research the most cutting edge AI applications of business productivity


In [16]:
#get list information and creates a dataframe for the rag model to reference. leaves description blank for now.

import requests
import pandas as pd

# Replace with your board ID
board_id = "5f1743793b184e31ee7d27f9"  # Example board ID

# Trello API URL for getting lists on a board
url = f"https://api.trello.com/1/boards/{board_id}/lists"

# Query parameters (API key and token)
query = {
    'key': 'd200138058c063cbb4ccd1c46bb572a0',  # Your API Key
    'token': 'ATTA2e7844b7cbb2557cf79ef8cd019d9f64f0fc77840bc4410149219a6f447c56b4D33239C4'  # Your API Token
}

# Make the request
response = requests.request("GET", url, params=query)

# Check if the request was successful
if response.status_code == 200:
    lists_data = response.json()
    
    # Convert JSON to DataFrame (no description included)
    df = pd.DataFrame(lists_data)
    
    # Select relevant columns (id and name)
    df = df[['id', 'name',]]  # Only keep id and name columns
    
    # Print the DataFrame
    print(df)
    
    # Creating a text context for the LLM (RAG model)
    context = "\n".join([f"ID: {row['id']}, Name: {row['name']}" for _, row in df.iterrows()])
    
    print("==== Context for RAG Model ====")
    print(context)
else:
    print(f"Error: {response.status_code}, {response.text}")

# Print the response for debugging
print(response.status_code)  # Check the status code (200 means success)
print(response.json())  # Print the JSON response (lists on the board)


                         id   name
0  5f1743793b184e31ee7d27fb  to Do
1  5f1743793b184e31ee7d27fc  Doing
2  5f1743793b184e31ee7d27fd   Done
==== Context for RAG Model ====
ID: 5f1743793b184e31ee7d27fb, Name: to Do
ID: 5f1743793b184e31ee7d27fc, Name: Doing
ID: 5f1743793b184e31ee7d27fd, Name: Done
200
[{'id': '5f1743793b184e31ee7d27fb', 'name': 'to Do', 'closed': False, 'color': None, 'idBoard': '5f1743793b184e31ee7d27f9', 'pos': 65535, 'subscribed': False, 'softLimit': None, 'type': None, 'datasource': {'filter': False}}, {'id': '5f1743793b184e31ee7d27fc', 'name': 'Doing', 'closed': False, 'color': None, 'idBoard': '5f1743793b184e31ee7d27f9', 'pos': 131071, 'subscribed': False, 'softLimit': None, 'type': None, 'datasource': {'filter': False}}, {'id': '5f1743793b184e31ee7d27fd', 'name': 'Done', 'closed': False, 'color': None, 'idBoard': '5f1743793b184e31ee7d27f9', 'pos': 196607, 'subscribed': False, 'softLimit': None, 'type': None, 'datasource': {'filter': False}}]


In [17]:
import pandas as pd
import asyncio
from beeai_framework.backend.chat import ChatModel
from beeai_framework.workflows.agent import AgentWorkflow, AgentWorkflowInput

# Define a class to encapsulate the results (list_id, card_name, card_description)
class TrelloCardDetails:
    def __init__(self, list_id, card_name, card_description):
        self.list_id = list_id
        self.card_name = card_name
        self.card_description = card_description

    def __str__(self):
        return f"TrelloCardDetails(list_id={self.list_id}, card_name={self.card_name}, card_description={self.card_description})"

# Initialize ChatModel for Ollama
async def main():
    # Initialize Ollama chat model
    llm = ChatModel.from_name("ollama:granite3.1-dense:8b")

    # Create context for the LLM by including list ID and Name
    context = "\n".join([f"ID: {row['id']}, Name: {row['name']}" for _, row in df.iterrows()])

    # Create the workflow
    workflow = AgentWorkflow(name="Smart assistant")

    # Add the CardNameExtractor agent
    workflow.add_agent(
        name="CardNameExtractor",
        role="A smart assistant who extracts the name of the card from the user input.",
        instructions="Extract the name of the card from the user input and return it as-is.",
        llm=llm,
    )

    # Add the DescriptionExtractor agent
    workflow.add_agent(
        name="DescriptionExtractor",
        role="A smart assistant who extracts the description of the card from the user input.",
        instructions="Extract the description of the card from the user input and return it as-is.",
        llm=llm,
    )

    # Add the ListRetriever agent
    workflow.add_agent(
        name="ListRetriever",
        role="A smart assistant who identifies the correct Trello list.",
        instructions="Match the user's description to one of the available Trello lists. Return ONLY the ID of the list.",
        llm=llm,
    )

    # Example of user input (make sure message is already defined with the transcription text)
    user_input = message

    # Run the workflow with inputs
    response = await workflow.run(
        inputs=[
            # First agent: Extract the card name
            AgentWorkflowInput(
                prompt=f"User input: '{user_input}'\n\nExtract the name of the card from the user input and return it as-is.",
                expected_output="Return the name of the card as-is.",
            ),
            # Second agent: Extract the description
            AgentWorkflowInput(
                prompt=f"User input: '{user_input}'\n\nExtract the description of the card from the user input and return it as-is.",
                expected_output="Return the description of the card as-is.",
            ),
            # Third agent: Identify the Trello list
            AgentWorkflowInput(
                prompt=f"Given the following list of Trello lists:\n{context}\n\nUser input: '{user_input}'\n\nWhich Trello list should the card be added to? Return ONLY the ID of the list.",
                expected_output="Return ONLY the ID of the list.",
            ),
        ]
    ).on(
        "success",
        lambda data, event: print(
            f"\n-> Step '{data.step}' has been completed with the following outcome.\n\n{data.state.final_answer}"
        ),
    )

    # Print the final answer
    print("==== Final Answer ====")
    print(response.result.final_answer)

    # Parse the outputs to extract list_id, card_name, and card_description
    try:
        # Extract card_name from the first agent's output
        card_name_output = response.steps[0].state.final_answer
        card_name_start = card_name_output.find('"') + 1  # Find the first quote and move to the next character
        card_name_end = card_name_output.rfind('"')  # Find the last quote
        card_name = card_name_output[card_name_start:card_name_end].strip()  # Extract text between quotes

        # Extract card_description from the second agent's output
        card_description_output = response.steps[1].state.final_answer
        card_description_start = card_description_output.find('"') + 1  # Find the first quote and move to the next character
        card_description_end = card_description_output.rfind('"')  # Find the last quote
        card_description = card_description_output[card_description_start:card_description_end].strip()  # Extract text between quotes

        # Extract list_id from the third agent's output
        list_id_output = response.steps[2].state.final_answer
        list_id = list_id_output.strip()  # The output should ONLY be the list ID

        # Validate that the list_id exists in the DataFrame
        if list_id not in df['id'].values:
            print(f"Error: List ID '{list_id}' not found in the DataFrame.")
            return None

        # Print the extracted values
        print(f"List ID: {list_id}")
        print(f"Card Name: {card_name}")
        print(f"Card Description: {card_description}")

        # Return the results as an object
        return TrelloCardDetails(list_id, card_name, card_description)

    except Exception as e:
        print("Error parsing the final answer:", e)
        return None

# Call the main function and capture the returned object
card_details = await main()

# Now you can use the object outside the main function
if card_details:
    print(f"List ID outside main: {card_details.list_id}")
    print(f"Card Name outside main: {card_details.card_name}")
    print(f"Card Description outside main: {card_details.card_description}")
else:
    print("No valid card details returned.")


-> Step 'CardNameExtractor' has been completed with the following outcome.

AI Productivity

-> Step 'DescriptionExtractor' has been completed with the following outcome.

research the most cutting edge AI applications of business productivity

-> Step 'ListRetriever' has been completed with the following outcome.

5f1743793b184e31ee7d27fb
==== Final Answer ====
5f1743793b184e31ee7d27fb
List ID: 5f1743793b184e31ee7d27fb
Card Name: AI Productivit
Card Description: research the most cutting edge AI applications of business productivit
List ID outside main: 5f1743793b184e31ee7d27fb
Card Name outside main: AI Productivit
Card Description outside main: research the most cutting edge AI applications of business productivit


In [18]:
import requests

# Assuming card_details is already defined from the previous cell
if card_details:
    # API endpoint for creating a Trello card
    url = "https://api.trello.com/1/cards"

    # Query parameters for the API request
    query = {
        'key': 'd200138058c063cbb4ccd1c46bb572a0',  # Your API Key
        'token': 'ATTA2e7844b7cbb2557cf79ef8cd019d9f64f0fc77840bc4410149219a6f447c56b4D33239C4',  # Your API Token
        'idList': card_details.list_id,  # Using the ID of the selected list
        'name': card_details.card_name,  # Name of the card (you can customize this)
        'desc': card_details.card_description  # Using the description of the selected list
    }

    # Make the request to create the card
    response = requests.post(url, params=query)

    # Print the response
    print("API Response Status Code:", response.status_code)  # Check the status code (200 means success)
    print("API Response JSON:", response.json())  # Print the JSON response with the created card details
else:
    print("No valid card details available to create a Trello card.")

API Response Status Code: 200
API Response JSON: {'id': '67e19ed3c887469e6c45c57c', 'badges': {'attachments': 0, 'fogbugz': '', 'checkItems': 0, 'checkItemsChecked': 0, 'checkItemsEarliestDue': None, 'comments': 0, 'description': True, 'due': None, 'dueComplete': False, 'lastUpdatedByAi': False, 'start': None, 'externalSource': None, 'attachmentsByType': {'trello': {'board': 0, 'card': 0}}, 'location': False, 'votes': 0, 'maliciousAttachments': 0, 'viewingMemberVoted': False, 'subscribed': False}, 'checkItemStates': [], 'closed': False, 'dueComplete': False, 'dateLastActivity': '2025-03-24T18:05:07.380Z', 'desc': 'research the most cutting edge AI applications of business productivit', 'descData': {'emoji': {}}, 'due': None, 'dueReminder': None, 'email': None, 'idBoard': '5f1743793b184e31ee7d27f9', 'idChecklists': [], 'idList': '5f1743793b184e31ee7d27fb', 'idMembers': [], 'idMembersVoted': [], 'idShort': 23, 'idAttachmentCover': None, 'labels': [], 'idLabels': [], 'manualCoverAttachmen